In [1]:
import tensorflow as tf
import datetime

In [2]:
tf.__version__

'2.0.0'

In [3]:
(train_image, train_labels),(test_image, test_labels) = tf.keras.datasets.mnist.load_data()

In [4]:
train_image.shape

(60000, 28, 28)

In [5]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [6]:
train_image.shape

(60000, 28, 28)

In [7]:
train_image = tf.expand_dims(train_image,-1)#扩展维度其他方法:1. tf.newx 2. napy里的reshape
test_image = tf.expand_dims(test_image,-1)

In [8]:
train_image.shape

TensorShape([60000, 28, 28, 1])

In [9]:
train_image = tf.cast(train_image/255, tf.float32)#归一化，转换数据类型
test_image = tf.cast(test_image/255, tf.float32)

In [10]:
train_labels = tf.cast(train_labels, tf.int64)#转换数据类型
test_labels = tf.cast(test_labels, tf.int64)

In [11]:
dataset = tf.data.Dataset.from_tensor_slices((train_image, train_labels))#扩展维度

In [12]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_image, test_labels))#扩展维度

In [13]:
dataset

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

In [14]:
dataset = dataset.repeat().shuffle(10000).batch(128) #crossfit方法需要重复repeat，自定义循环不需要repeat
test_dataset = test_dataset.repeat().batch(128)

In [15]:
dataset

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

In [16]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, [3,3],activation='relu',input_shape=(None,None,1)),
    tf.keras.layers.Conv2D(32,[3,3], activation='relu'),
    tf.keras.layers.GlobalMaxPooling2D(),#全局平均池化
    tf.keras.layers.Dense(10, activation='softmax')
])

In [17]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [18]:
import os
log_dir = os.path.join('logs', datetime.datetime.now().strftime("%Y%m%d-%H%M%S")) #日志目录添加时间

In [19]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir, histogram_freq=1)  #histogram_freq参数是直方图，记录参数变量的ping率

In [20]:
file_writer = tf.summary.create_file_writer(log_dir + '/lr') #文件编写器

In [21]:
file_writer.set_as_default() #设为默认文件编写器

In [22]:
def lr_sche(epoch):
    learning_rate = 0.2
    if epoch > 5:
        learning_rate = 0.02
    if epoch > 10:
        learning_rate = 0.01
    if epoch > 20:
        learning_rate = 0.005
    tf.summary.scalar('leaning_rate',data=learning_rate, step=epoch)
    return learning_rate        

In [23]:
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_sche)

In [24]:
model.fit(dataset,
         epochs=25,
         steps_per_epoch=60000//128,
         validation_data=test_dataset,
         validation_steps=1000//128,
          callbacks=[tensorboard_callback, lr_callback] #回调函数
         )

Train for 468 steps, validate for 7 steps
Epoch 1/25
468/468 [==============================] - 21s 44ms/step - loss: 2.3513 - accuracy: 0.1043 - val_loss: 2.3275 - val_accuracy: 0.0993
Epoch 2/25
468/468 [==============================] - 19s 40ms/step - loss: 2.3136 - accuracy: 0.1027 - val_loss: 2.3062 - val_accuracy: 0.0993
Epoch 3/25
468/468 [==============================] - 19s 41ms/step - loss: 2.3120 - accuracy: 0.1054 - val_loss: 2.3119 - val_accuracy: 0.1105
Epoch 4/25
468/468 [==============================] - 19s 41ms/step - loss: 2.3140 - accuracy: 0.1038 - val_loss: 2.3066 - val_accuracy: 0.0859
Epoch 5/25
468/468 [==============================] - 19s 41ms/step - loss: 2.3134 - accuracy: 0.1018 - val_loss: 2.3185 - val_accuracy: 0.0993
Epoch 6/25
468/468 [==============================] - 19s 41ms/step - loss: 2.3131 - accuracy: 0.1028 - val_loss: 2.3029 - val_accuracy: 0.1228
Epoch 7/25
468/468 [==============================] - 20s 42ms/step - loss: 2.3027 - accuracy:

In [25]:
#8.2 

In [26]:
#显示在notebook中

In [27]:
%load_ext tensorboard
%matplotlib inline

In [31]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 3784), started 18:25:48 ago. (Use '!kill 3784' to kill it.)

In [29]:
# cmd 进入命令窗口输入 tensorboard --logdir logs

In [30]:
#8.4